In [1]:
# Imports necesarios
import numpy as np
import pandas as pd
#import seaborn as sb
import matplotlib.pyplot as plt
%matplotlib inline
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
plt.rcParams['figure.figsize'] = (16, 9)
plt.style.use('ggplot')
#from sklearn import linear_model
#from sklearn.metrics import mean_squared_error, r2_score
#from sklearn.neural_network import MLPRegressor

## Abrir Archivos

In [2]:

#cargamos los datos de entrada
data_train = pd.read_csv("/Users/manuelmoya/Documents/USACH/2019-1er/Tesis/1.Desarrollo/05.AnalisisDatos/Datos/train.csv",header=0)
data_wf1 = pd.read_csv("/Users/manuelmoya/Documents/USACH/2019-1er/Tesis/1.Desarrollo/05.AnalisisDatos/Datos/wf1.csv",header=0)

#veamos cuantas dimensiones y registros contiene
data_train.shape

# Ver los 1ros registros
data_train.head()

,date,wp1,wp2,wp3,wp4,wp5,wp6,wp7
0,2009070100,0.045,0.233,0.494,0.105,0.056,0.118,0.051
1,2009070101,0.085,0.249,0.257,0.105,0.066,0.066,0.051
2,2009070102,0.020,0.175,0.178,0.033,0.015,0.026,0.000
3,2009070103,0.060,0.085,0.109,0.022,0.010,0.013,0.000
4,2009070104,0.045,0.032,0.079,0.039,0.010,0.000,0.000


In [3]:
data_wf1.head()

,date,hors,u,v,ws,wd
0,2009070100,1,2.34,-0.79,2.47,108.68
1,2009070100,2,2.18,-0.99,2.40,114.31
2,2009070100,3,2.20,-1.21,2.51,118.71
3,2009070100,4,2.35,-1.40,2.73,120.86
4,2009070100,5,2.53,-1.47,2.93,120.13


## Agregar campos auxiliares

In [4]:
# agregar anno
def anno(num):
    num_str = str(num)
    num_corto = num_str[0:4]
    return int(num_corto)

# agregar mes
def mes(num):
    num_str = str(num)
    num_corto = num_str[4:6]
    return int(num_corto)

# agregar mes
def dia(num):
    num_str = str(num)
    num_corto = num_str[6:8]
    return int(num_corto)

# agregar hrsObs
def horsObs(num):
    num_str = str(num)
    num_corto = num_str[8:10]
    return int(num_corto)

def upd_date(num):
    num_str = str(num)
    num_corto = num_str[0:8]
    return int(num_corto)


# Crear campos
data_wf1['anno'] = data_wf1['date'].apply(anno)
data_wf1['mes'] = data_wf1['date'].apply(mes)
data_wf1['dia'] = data_wf1['date'].apply(dia)
data_wf1['horsObs'] = data_wf1['date'].apply(horsObs)

# Actualizar date
data_wf1['date'] = data_wf1['date'].apply(upd_date)

data_wf1['daysAdd'] = -5
data_wf1['horsFore']= -5
data_wf1['dateFore']= -5
data_wf1['wp1']= -5




data_wf1

,date,hors,u,v,ws,wd,anno,mes,dia,horsObs,daysAdd,horsFore,dateFore,wp1
0,20090701,1,2.34,-0.79,2.47,108.68,2009,7,1,0,-5,-5,-5,-5
1,20090701,2,2.18,-0.99,2.40,114.31,2009,7,1,0,-5,-5,-5,-5
2,20090701,3,2.20,-1.21,2.51,118.71,2009,7,1,0,-5,-5,-5,-5
3,20090701,4,2.35,-1.40,2.73,120.86,2009,7,1,0,-5,-5,-5,-5
4,20090701,5,2.53,-1.47,2.93,120.13,2009,7,1,0,-5,-5,-5,-5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
104827,20120626,44,2.44,1.08,2.67,66.14,2012,6,26,12,-5,-5,-5,-5
104828,20120626,45,2.11,1.46,2.57,55.39,2012,6,26,12,-5,-5,-5,-5
104829,20120626,46,1.63,1.61,2.29,45.42,2012,6,26,12,-5,-5,-5,-5
104830,20120626,47,1.07,1.58,1.91,34.05,2012,6,26,12,-5,-5,-5,-5


## Actualizar campos auxiliares

In [5]:
# daysAdd
def asigna_daysAdd(hors, horsObs):
    suma=0
    if horsObs == 0:
        if hors > 24:
           suma=1           
    else:
        if (hors > 12) & (hors <= 36):
          suma=1
        elif hors > 36: 
          suma=2
    return suma

data_wf1['daysAdd'] = data_wf1.apply(lambda x: asigna_daysAdd(x['hors'],x['horsObs']), axis=1);

In [10]:
# horsFore
def asigna_horsFore(hors, horsObs):
    hrs=hors
    if horsObs == 0:
        if hors > 24:
           hrs=hors-24         
    else:
        if (hors > 12) & (hors <= 36):
          hrs=hors-12 
        elif hors > 36: 
          hrs=hors-36
    return int(hrs)

data_wf1['horsFore'] = data_wf1.apply(lambda x: asigna_horsFore(x['hors'],x['horsObs']), axis=1);

In [13]:
# dateFore
from datetime import timedelta,datetime

# Convertir num 2 digitos con cero a la izquierda
def conv_num_2D(num):
  num_conv = "0" + str(num)
  num_final = num_conv[len(num_conv)-2:len(num_conv)]
  
  return num_final


# Asignar dateFore
def asigna_dateFore(anno, mes, dia,daysAdd, horsFore):
    fecha = str(int(anno)) +"-"+ str(int(mes)) +"-"+ str(int(dia))
    datetime_obj = datetime.strptime(fecha, '%Y-%m-%d')
    fecha_fin = datetime_obj + timedelta(days=daysAdd)
    #date_time = datetime.fromtimestamp(fecha_fin)
    fecha_format= fecha_fin.strftime("%Y%m%d")
    fecha_format = fecha_format + conv_num_2D(horsFore)
    
    return int(fecha_format)

data_wf1['dateFore'] = data_wf1.apply(lambda x: asigna_dateFore(x['anno'],x['mes'],x['dia'],x['daysAdd'],x['horsFore']), axis=1);
data_wf1

,date,hors,u,v,ws,wd,anno,mes,dia,horsObs,daysAdd,horsFore,dateFore,wp1
0,20090701,1,2.34,-0.79,2.47,108.68,2009,7,1,0,0,1,2009070101,-5
1,20090701,2,2.18,-0.99,2.40,114.31,2009,7,1,0,0,2,2009070102,-5
2,20090701,3,2.20,-1.21,2.51,118.71,2009,7,1,0,0,3,2009070103,-5
3,20090701,4,2.35,-1.40,2.73,120.86,2009,7,1,0,0,4,2009070104,-5
4,20090701,5,2.53,-1.47,2.93,120.13,2009,7,1,0,0,5,2009070105,-5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
104827,20120626,44,2.44,1.08,2.67,66.14,2012,6,26,12,2,8,2012062808,-5
104828,20120626,45,2.11,1.46,2.57,55.39,2012,6,26,12,2,9,2012062809,-5
104829,20120626,46,1.63,1.61,2.29,45.42,2012,6,26,12,2,10,2012062810,-5
104830,20120626,47,1.07,1.58,1.91,34.05,2012,6,26,12,2,11,2012062811,-5


In [14]:
# Asignar wp1
def asigna_wp1(dateFore):
    busq_wp1 = data_train['wp1'][ (data_train['date'] ==  dateFore)]
    
    val_final= pd.NA
    if len(busq_wp1)>0:
         val_final=busq_wp1.values[0]
    
    return val_final

data_wf1['wp1'] = data_wf1.apply(lambda x: asigna_wp1(x['dateFore']), axis=1);
data_wf1

,date,hors,u,v,ws,wd,anno,mes,dia,horsObs,daysAdd,horsFore,dateFore,wp1
0,20090701,1,2.34,-0.79,2.47,108.68,2009,7,1,0,0,1,2009070101,0.085
1,20090701,2,2.18,-0.99,2.40,114.31,2009,7,1,0,0,2,2009070102,0.02
2,20090701,3,2.20,-1.21,2.51,118.71,2009,7,1,0,0,3,2009070103,0.06
3,20090701,4,2.35,-1.40,2.73,120.86,2009,7,1,0,0,4,2009070104,0.045
4,20090701,5,2.53,-1.47,2.93,120.13,2009,7,1,0,0,5,2009070105,0.035
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
104827,20120626,44,2.44,1.08,2.67,66.14,2012,6,26,12,2,8,2012062808,NaN
104828,20120626,45,2.11,1.46,2.57,55.39,2012,6,26,12,2,9,2012062809,NaN
104829,20120626,46,1.63,1.61,2.29,45.42,2012,6,26,12,2,10,2012062810,NaN
104830,20120626,47,1.07,1.58,1.91,34.05,2012,6,26,12,2,11,2012062811,NaN


## Eliminar columnas auxiliares

In [19]:
# BOrrar Filas con NaN
frm_final = data_wf1.dropna() 
frm_final

,date,hors,u,v,ws,wd,anno,mes,dia,horsObs,daysAdd,horsFore,dateFore,wp1
0,20090701,1,2.34,-0.79,2.47,108.68,2009,7,1,0,0,1,2009070101,0.085
1,20090701,2,2.18,-0.99,2.40,114.31,2009,7,1,0,0,2,2009070102,0.02
2,20090701,3,2.20,-1.21,2.51,118.71,2009,7,1,0,0,3,2009070103,0.06
3,20090701,4,2.35,-1.40,2.73,120.86,2009,7,1,0,0,4,2009070104,0.045
4,20090701,5,2.53,-1.47,2.93,120.13,2009,7,1,0,0,5,2009070105,0.035
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
104791,20120626,8,4.19,-6.42,7.67,146.86,2012,6,26,12,0,8,2012062608,0.17
104792,20120626,9,4.17,-6.09,7.38,145.59,2012,6,26,12,0,9,2012062609,0.211
104793,20120626,10,4.08,-4.92,6.40,140.34,2012,6,26,12,0,10,2012062610,0.251
104794,20120626,11,3.92,-3.38,5.18,130.76,2012,6,26,12,0,11,2012062611,0.301


In [20]:
frm_final

,date,hors,u,v,ws,wd,anno,mes,dia,horsObs,daysAdd,horsFore,dateFore,wp1
0,20090701,1,2.34,-0.79,2.47,108.68,2009,7,1,0,0,1,2009070101,0.085
1,20090701,2,2.18,-0.99,2.40,114.31,2009,7,1,0,0,2,2009070102,0.02
2,20090701,3,2.20,-1.21,2.51,118.71,2009,7,1,0,0,3,2009070103,0.06
3,20090701,4,2.35,-1.40,2.73,120.86,2009,7,1,0,0,4,2009070104,0.045
4,20090701,5,2.53,-1.47,2.93,120.13,2009,7,1,0,0,5,2009070105,0.035
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
104791,20120626,8,4.19,-6.42,7.67,146.86,2012,6,26,12,0,8,2012062608,0.17
104792,20120626,9,4.17,-6.09,7.38,145.59,2012,6,26,12,0,9,2012062609,0.211
104793,20120626,10,4.08,-4.92,6.40,140.34,2012,6,26,12,0,10,2012062610,0.251
104794,20120626,11,3.92,-3.38,5.18,130.76,2012,6,26,12,0,11,2012062611,0.301


In [21]:
frm_final.drop(['u','v','daysAdd','horsFore','dateFore'], axis=1, inplace=True)

/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [22]:
frm_final

,date,hors,ws,wd,anno,mes,dia,horsObs,wp1
0,20090701,1,2.47,108.68,2009,7,1,0,0.085
1,20090701,2,2.40,114.31,2009,7,1,0,0.02
2,20090701,3,2.51,118.71,2009,7,1,0,0.06
3,20090701,4,2.73,120.86,2009,7,1,0,0.045
4,20090701,5,2.93,120.13,2009,7,1,0,0.035
...,...,...,...,...,...,...,...,...,...
104791,20120626,8,7.67,146.86,2012,6,26,12,0.17
104792,20120626,9,7.38,145.59,2012,6,26,12,0.211
104793,20120626,10,6.40,140.34,2012,6,26,12,0.251
104794,20120626,11,5.18,130.76,2012,6,26,12,0.301


In [23]:

frm_final.to_csv('/Users/manuelmoya/Documents/USACH/2019-1er/Tesis/1.Desarrollo/05.AnalisisDatos/Datos/wf1_serie.csv')
